In [13]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd

In [19]:
url = "https://www.pdga.com/tour/event/87822"
page = get(url)
event_id = url.split("/")[-1]
soup = BeautifulSoup(page.content, "html.parser")

In [16]:
# first lets get tourny level info
event_name = soup.find('h1', attrs={'id':"page-title"}).text.replace('/','_')
event_info = soup.find_all("div", attrs={"class":"pane-tournament-event-info"})[0]
event_date = event_info.find('li', attrs={'class':"tournament-date"}).text.split(':')[1].strip()
event_location = event_info.find('li', attrs={'class':"tournament-location"}).text.split(":")[1].strip()
event_city, event_state, event_country = event_location.split(",")
event_director = event_info.find('li', attrs={'class': "tournament-director"}).text.split(":")[1].strip()
event_type = event_info.find("h4").text
event_view = soup.find_all("div", attrs={"class":"pane-tournament-event-view"})[0]
event_status = event_view.find("td", attrs={"class":"status"}).text
print(event_status)
# if event_status != "Event complete; official ratings processed":
#     # event not finalized, dont gather data
#     return 0
event_player_count = event_view.find("td", attrs={"class":"players"}).text
try:
    event_purse = event_view.find("td", attrs={"class":"purse"}).text
except AttributeError:
    event_purse = None

Sanctioned


In [ ]:
# first lets get tourny level info
records = []
event_name = soup.find('h1', attrs={'id':"page-title"}).text.replace('/','_')
event_info = soup.find_all("div", attrs={"class":"pane-tournament-event-info"})[0]
event_date = event_info.find('li', attrs={'class':"tournament-date"}).text.split(':')[1].strip()
event_location = event_info.find('li', attrs={'class':"tournament-location"}).text.split(":")[1].strip()
event_city, event_state, event_country = event_location.split(",")
event_director = event_info.find('li', attrs={'class': "tournament-director"}).text.split(":")[1].strip()
event_type = event_info.find("h4").text
event_view = soup.find_all("div", attrs={"class":"pane-tournament-event-view"})
if event_view is not None:
    event_status = event_view[0].find("td", attrs={"class":"status"}).text
    if event_status != "Event complete; official ratings processed":
        # event not finalized, dont gather data
        print('Event still pending', event_id)
    event_player_count = event_view[0].find("td", attrs={"class":"players"}).text
    try:
        event_purse = event_view.find("td", attrs={"class":"purse"}).text
    except AttributeError:
        event_purse = ""
else:
    event_player_count = ""
    event_purse = ""

#find each of the divisions in the event
divisions = soup.find_all('details')
for division in divisions:
    # each division can play at a different location/layout
    div_id = division.find("h3")["id"]
    rounds = division.find_all("th", attrs={"class":"round"})
    for round in rounds:
        # each round can be played at a different location/layout
        round_id = round.text.strip("Rd")
        round_info = division.find("div", attrs={"id":f"layout-details-{event_id}-{div_id}-round-{round_id}"}).text
        round_course = round_info.split(";")[0].split("-")[0].strip()
        round_layout = round_info.split(";")[0].split("-")[1].strip()
        round_holes = round_info.split(";")[1].split()[0].strip()
        round_par = round_info.split(";")[2].split()[1].strip()
        round_dist = round_info.split(";")[3].strip()
        # get each players score for this round
        players = division.find_all("tr")
        for player in players:
            if "th" in player.contents[0].name:
                continue
            try:
                player_points = player.find("td", attrs={"class":"points"}).text
            except AttributeError:
                player_points = ""
            player_pdga = player.find("td", attrs={"class":"pdga-number"}).text
            player_rating = player.find("td", attrs={"class":"player-rating"}).text
            player_round_score = player.find_all("td", attrs={"class":"round"})[int(round_id)-1].text
            player_round_rating = player.find_all("td", attrs={"class":"round-rating"})[int(round_id)-1].text
            # add all the info to the record list
            records.append([
                event_name,
                event_date,
                event_city,
                event_state,
                event_country,
                event_director,
                event_type,
                event_player_count,
                event_purse,
                div_id,
                round_id,
                round_course,
                round_layout,
                round_holes,
                round_par,
                round_dist,
                player_pdga,
                player_points,
                player_rating,
                player_round_score,
                player_round_rating
            ])

df = pd.DataFrame(columns=[
    "event_name",
    "event_date",
    "event_city",
    "event_state",
    "event_country",
    "event_director",
    "event_type",
    "event_player_count",
    "event_purse",
    "event_division",
    "round_number",
    "round_course",
    "round_layout",
    "layout_holes",
    "layout_par",
    "layout_distance",
    "player_pdga",
    "player_earned_points",
    "player_rating",
    "player_round_score",
    "player_round_rating"
], data=records)
df

Event still pending 87822


,event_name,event_date,event_city,event_state,event_country,event_director,event_type,event_player_count,event_purse,event_division,...,round_course,round_layout,layout_holes,layout_par,layout_distance,player_pdga,player_earned_points,player_rating,player_round_score,player_round_rating
0,6th Annual Snowy Eagle Open,22-Feb-2025,Chambersburg,Pennsylvania,United States,Asher Miller,Pro/Am C-Tier,61,None,MPO,...,Cove Valley Christian Youth Camp,Short Tees,18,58,"5,572 ft.",81781,None,1008,50,1009
1,6th Annual Snowy Eagle Open,22-Feb-2025,Chambersburg,Pennsylvania,United States,Asher Miller,Pro/Am C-Tier,61,None,MPO,...,Cove Valley Christian Youth Camp,Short Tees,18,58,"5,572 ft.",144433,None,988,53,979
2,6th Annual Snowy Eagle Open,22-Feb-2025,Chambersburg,Pennsylvania,United States,Asher Miller,Pro/Am C-Tier,61,None,MPO,...,Cove Valley Christian Youth Camp,Short Tees,18,58,"5,572 ft.",268515,None,918,58,930
3,6th Annual Snowy Eagle Open,22-Feb-2025,Chambersburg,Pennsylvania,United States,Asher Miller,Pro/Am C-Tier,61,None,MPO,...,Cove Valley Christian Youth Camp,Short Tees,18,58,"5,572 ft.",27920,None,974,57,940
4,6th Annual Snowy Eagle Open,22-Feb-2025,Chambersburg,Pennsylvania,United States,Asher Miller,Pro/Am C-Tier,61,None,MPO,...,Cove Valley Christian Youth Camp,Short Tees,18,58,"5,572 ft.",104782,None,975,59,920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,6th Annual Snowy Eagle Open,22-Feb-2025,Chambersburg,Pennsylvania,United States,Asher Miller,Pro/Am C-Tier,61,None,MA4,...,Cove Valley Christian Youth Camp,Long Tees,18,65,"7,606 ft.",152902,None,825,75,878
118,6th Annual Snowy Eagle Open,22-Feb-2025,Chambersburg,Pennsylvania,United States,Asher Miller,Pro/Am C-Tier,61,None,MA4,...,Cove Valley Christian Youth Camp,Long Tees,18,65,"7,606 ft.",270533,None,827,84,807
119,6th Annual Snowy Eagle Open,22-Feb-2025,Chambersburg,Pennsylvania,United States,Asher Miller,Pro/Am C-Tier,61,None,MA4,...,Cove Valley Christian Youth Camp,Long Tees,18,65,"7,606 ft.",246259,None,816,91,751
120,6th Annual Snowy Eagle Open,22-Feb-2025,Chambersburg,Pennsylvania,United States,Asher Miller,Pro/Am C-Tier,61,None,MA4,...,Cove Valley Christian Youth Camp,Long Tees,18,65,"7,606 ft.",263198,None,842,93,736


In [9]:
records = []

In [10]:
#find each of the division results
divisions = soup.find_all('details')
for division in divisions:
    # each division can play at a different location/layout
    div_id = division.find("h3")["id"]
    rounds = division.find_all("th", attrs={"class":"round"})
    for round in rounds:
        # each round can be played at a different location/layout
        round_id = round.text.strip("Rd")
        round_info = division.find("div", attrs={"id":f"layout-details-{event_id}-{div_id}-round-{round_id}"}).text
        round_course = round_info.split(";")[0].split("-")[0].strip()
        round_layout = round_info.split(";")[0].split("-")[1].strip()
        round_holes = round_info.split(";")[1].split()[0].strip()
        round_par = round_info.split(";")[2].split()[1].strip()
        round_dist = round_info.split(";")[3].strip()
        # get each players score for this round
        players = division.find_all("tr")
        for player in players:
            if "th" in player.contents[0].name:
                continue
            player_points = player.find("td", attrs={"class":"points"}).text
            player_pdga = player.find("td", attrs={"class":"pdga-number"}).text
            player_rating = player.find("td", attrs={"class":"player-rating"}).text
            player_round_score = player.find_all("td", attrs={"class":"round"})[int(round_id)-1].text
            player_round_rating = player.find_all("td", attrs={"class":"round-rating"})[int(round_id)-1].text
            records.append([
                event_name,
                event_date,
                event_city,
                event_state,
                event_country,
                event_director,
                event_type,
                event_player_count,
                event_purse,
                div_id,
                round_id,
                round_course,
                round_layout,
                round_holes,
                round_par,
                round_dist,
                player_pdga,
                player_points,
                player_rating,
                player_round_score,
                player_round_rating
            ])

In [11]:
df = pd.DataFrame(columns=[
    "event_name",
    "event_date",
    "event_city",
    "event_state",
    "event_country",
    "event_director",
    "event_type",
    "event_player_count",
    "event_purse",
    "event_division",
    "round_number",
    "round_course",
    "round_layout",
    "layout_holes",
    "layout_par",
    "layout_distance",
    "player_pdga",
    "player_earned_points",
    "player_rating",
    "player_round_score",
    "player_round_rating"
], data=records)
df

,event_name,event_date,event_city,event_state,event_country,event_director,event_type,event_player_count,event_purse,event_division,...,round_course,round_layout,layout_holes,layout_par,layout_distance,player_pdga,player_earned_points,player_rating,player_round_score,player_round_rating
0,Hickory Run Upshot,08-Feb-2025,White Haven,Pennsylvania,United States,Brian Bochantin,Pro/Am C-Tier,41,$112,MPO,...,Hickory Run State Park,BS Upshot Icy Long,18,54,"4,835 ft.",99158,30.00,983,48,1010
1,Hickory Run Upshot,08-Feb-2025,White Haven,Pennsylvania,United States,Brian Bochantin,Pro/Am C-Tier,41,$112,MPO,...,Hickory Run State Park,BS Upshot Icy Long,18,54,"4,835 ft.",26126,25.00,981,54,945
2,Hickory Run Upshot,08-Feb-2025,White Haven,Pennsylvania,United States,Brian Bochantin,Pro/Am C-Tier,41,$112,MPO,...,Hickory Run State Park,BS Upshot Icy Long,18,54,"4,835 ft.",197899,20.00,917,52,967
3,Hickory Run Upshot,08-Feb-2025,White Haven,Pennsylvania,United States,Brian Bochantin,Pro/Am C-Tier,41,$112,MPO,...,Hickory Run State Park,BS Upshot Icy Long,18,54,"4,835 ft.",190907,15.00,932,53,956
4,Hickory Run Upshot,08-Feb-2025,White Haven,Pennsylvania,United States,Brian Bochantin,Pro/Am C-Tier,41,$112,MPO,...,Hickory Run State Park,BS Upshot Icy Long,18,54,"4,835 ft.",201333,10.00,922,57,913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,Hickory Run Upshot,08-Feb-2025,White Haven,Pennsylvania,United States,Brian Bochantin,Pro/Am C-Tier,41,$112,MA4,...,Hickory Run State Park,BS Upshot Icy Short,18,56,"4,578 ft.",295265,10.00,824,61,843
78,Hickory Run Upshot,08-Feb-2025,White Haven,Pennsylvania,United States,Brian Bochantin,Pro/Am C-Tier,41,$112,MA4,...,Hickory Run State Park,BS Upshot Icy Short,18,56,"4,578 ft.",293219,8.00,796,68,762
79,Hickory Run Upshot,08-Feb-2025,White Haven,Pennsylvania,United States,Brian Bochantin,Pro/Am C-Tier,41,$112,MA4,...,Hickory Run State Park,BS Upshot Icy Short,18,56,"4,578 ft.",295676,6.00,721,67,773
80,Hickory Run Upshot,08-Feb-2025,White Haven,Pennsylvania,United States,Brian Bochantin,Pro/Am C-Tier,41,$112,MA4,...,Hickory Run State Park,BS Upshot Icy Short,18,56,"4,578 ft.",207573,0.00,783,999,


In [79]:
for course, course_df in df.groupby("round_course"):
    print(course)

Covered Bridge Park
